# EXAMPLE TICKER

In [1]:
ticker = 'CVX'

# GET DATA

In [2]:
import pandas as pd
import yfinance as yf
from pandas_datareader import DataReader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# read annual original Fama-French factors from 1926 and compute means
ff= pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]
fprem = ff[['Mkt-RF','SMB','HML']].mean()

# read CMA and RMW from 1964, compute means, and combine with other means
ff= pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=1964)[1]
fprem = pd.concat((fprem,ff[['RMW','CMA']].mean()))


# read monthly Fama-French factors for past 60 months
ff = pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=2017)[0]
ff = ff.iloc[-60:]

# get current 3-month Treasury bill yield from FRED
rf = pdr('DGS3MO','fred',start=2022).iloc[-1].item()

# pull data from yahoo for ticker
ticker = ticker.upper()
ret = yf.download(ticker, start='2017-01-01', progress=False)['Adj Close']
ret = ret.resample('M').last().pct_change()*100
# ret.index = ret.index.to_period('M')
ret.name = "ret"

# merge with French data, reducing to 60 months in the process
ret.index = ret.index.to_period('M')
df = ff.join(ret,how='left')

# compute excess returns for the ticker 
df['ret'] -= df.RF

# RUN TIME-SERIES REGRESSION

In [3]:
import statsmodels.api as sm

# run regression of excess return on market excess return and get beta
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
result = sm.OLS(df.ret,sm.add_constant(df[factors])).fit()
betas = result.params[factors]

# CALCULATE COST OF EQUITY

In [4]:
# compute cost of equity
eret = rf + betas@fprem


print('Factor Risk Premia and Betas:')
for i,f in enumerate(factors):
    print(f'\tFactor:\t{f}\tRisk Premium:\t{fprem[i]:.2f}\tBeta\t{betas[i]:.2f}')
print(f'{ticker} Risk Premium:\t{betas@fprem:.2f}')
print(f'Risk-free Rate:\t\t{rf:.2f}')
print(f'{ticker} Cost of Equity:\t{eret:.2f}')

Factor Risk Premia and Betas:
	Factor:	Mkt-RF	Risk Premium:	8.55	Beta	1.25
	Factor:	SMB	Risk Premium:	2.91	Beta	-0.01
	Factor:	HML	Risk Premium:	4.43	Beta	0.64
	Factor:	RMW	Risk Premium:	3.54	Beta	-0.38
	Factor:	CMA	Risk Premium:	3.76	Beta	0.85
CVX Risk Premium:	15.37
Risk-free Rate:		5.62
CVX Cost of Equity:	20.99
